        Prototyping notebook for predicting stock volaitility, prices, etc using extra data from web trends, news, etc. 

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
pd.set_option('io.parquet.engine', 'pyarrow')
import numpy as np
import os
import gc, os
from getpass import getpass
from utils import *
from models import *
import gc
import shutil
import zipfile
import torch
import arrow
import matplotlib.pyplot as plt
import torch



# currently hardcoded to use GPU in order to identify when cuda is installed incorrectly. The models will not be practical to train on CPU
cuda = torch.device("cuda") 
cpu = torch.device("cpu")
# if you don't need the API downloads, you can set this to False
use_api = False

38
seed=42
np.random.seed(seed)
torch.manual_seed(seed)
rng = np.random.default_rng(seed=seed) 
MODEL = 'cardiffnlp/twitter-xlm-roberta-base-sentiment' #cardiffnlp/twitter-xlm-roberta-base-sentiment  cardiffnlp/twitter-roberta-base-emotion
print('Available GPU memory:', available_mem(), 'GB')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 1: Enter password to save/load API keys. This will be used to create/subsequently unlock the encrypted API keys.
# 2: if it is the first time running this, this will ask you to enter your username + a single space + your Kaggle API key. (don't use quotes)
# 3: any other api keys in future will follow the same format

# # may take awhile to load the first time, depending on your internet speeds. 
# After this first run, you will only need to enter the password to load the api keys 
# NOTE: Don't delete or move salt.secret as all tokens will become undecryptable. 
# If you made a mistake or need to retry, you may 1) delete salt.secret to reset everything. 2) delete the specific .secret key to re-enter only that info.
if use_api:
    # Ask for input of password to save API keys
    password = getpass("Enter password to save/load API keys: ");
    if len(password) > 0:
        kaggle_api_key = fernet_key_encryption(password, 'Kaggle');
        #td_ameritrade_api_key = fernet_key_encryption(password, 'TD_Ameritrade')
        #data_nasdaq_key = fernet_key_encryption(password, 'Nasdaq');
        del password;
        gc.collect();
        get_datasets(kaggle_api_key);
    elif len(glob('data/*')) < 4:
        cont = input(f"Password is empty. Press n to cancel or any other key to continue: ")
        if cont == 'n':
            assert False, "Exiting program. Please enter a password to continue."

In [ ]:
# gathers a dataframe of useful terms and info related to every stock in data\Stock_List.parquet. This may take up to 40 minutes to run the first time (the web scraping is slow due to requests being throttled). Stores results in company_list.pkl
search_terms = aquire_stock_search_terms('data/Stock/')

In [ ]:
update_stock_data(ticker='KR', save = True)
update_stock_data(ticker='AAPL', save = True)
update_stock_data(ticker='TSLA', save = True)

In [ ]:
if False:
    search = search_terms.data[search_terms.data['ticker'] == 'KR'].values.tolist()[0][0:4]
    search = [x.lower() for x in search if x != None and x != '']
    search[0] = '#' + search[0]
    df_if_error = scrape_tweets(since='2018-01-01', until='2023-04-19', max_tweets=20, update_twitter_data=True, co_list=search)

    search = search_terms.data[search_terms.data['ticker'] == 'AAPL'].values.tolist()[0][0:4]
    search = [x.lower() for x in search if x != None and x != '']
    search[0] = '#' + search[0]
    df_if_error = scrape_tweets(since='2018-01-01', until='2023-04-19', max_tweets=20, update_twitter_data=True, co_list=search)


    # search = search_terms.data[search_terms.data['ticker'] == 'TSLA'].values.tolist()[0][0:4]
    # search = [x.lower() for x in search if x != None and x != '']
    # search[0] = '#' + search[0]
    # df_if_error = scrape_tweets(since='2018-01-01', until='2023-04-19', max_tweets=20, update_twitter_data=True, co_list=search)
    

# how to recover the data if the crawler crashes (example for apple inc.)
#tweets_df = pd.DataFrame(df_if_error, columns=['date', 'text', 'username', 'searchterm'])
#tweets_df = tweets_df.drop_duplicates(inplace=False, subset=['date', 'text', 'username', 'searchterm']).reset_index(drop=True, inplace=False).dropna(inplace=False)
#save_file(tweets_df, r'data\Twitter\twitter_data___#aapl---apple inc.---timothy cook---consumer electronics.parquet')

In [ ]:
emotion_classifier = emotion_classifier_load(MODEL)
text_df = classify_twitter_text(save_path='data/Text/text_emotion_29.parquet', model=emotion_classifier, load_path='data/Text/text_emotion_29.parquet')
text_df = text_df.drop(columns=['stock','text'], inplace=False).dropna(inplace=False)
# len date row chars > 7
text_df = text_df[text_df['date'].str.len() > 7]
text_df


In [ ]:
index = glob('data/Twitter/twitter_emotion___#*.parquet')
index = {x.split("#")[1].split("---")[0]: x for x in index}

kr_text =   load_file(index['kr'].replace('emotion', 'data'))
aapl_text = load_file(index['aapl'].replace('emotion', 'data'))

kr_df   = classify_twitter_text(save_path=index['kr'], model=emotion_classifier, load_path=index['kr'].replace('emotion', 'data'))
aapl_df = classify_twitter_text(save_path=index['aapl'], model=emotion_classifier, load_path=index['aapl'].replace('emotion', 'data'))
aapl_df

In [ ]:
# kr stock
stock_kr = load_file('data/Stock/KR.parquet')
#kr_df, stock_kr = intersect_df(kr_df, stock_kr)
kr_df['date'] = kr_df['date'].apply(lambda x: arrow.get(str(x)[:10]).format('YYYY-MM-DD') if len(str(x)) > 7 else None).dropna()
#kr_df, stock_kr = intersect_df(kr_df, stock_kr)
kr_stats = daily_stats(kr_df)

stock_kr = load_file('data/Stock/KR.parquet')
stock_kr, text_df_kr = intersect_df(stock_kr, text_df)
text_stats_kr = daily_stats(text_df_kr)
text_df_kr = daily_stats(text_df_kr)



# aapl stock
stock_aapl = load_file('data/Stock/AAPL.parquet')
#aapl_df, stock_aapl = intersect_df(aapl_df, stock_aapl)
aapl_df['date'] = aapl_df['date'].apply(lambda x: arrow.get(str(x)[:10]).format('YYYY-MM-DD') if len(str(x)) > 7 else None).dropna()
#aapl_df, stock_aapl = intersect_df(aapl_df, stock_aapl)
aapl_stats = daily_stats(aapl_df)

stock_aapl = load_file('data/Stock/AAPL.parquet')
stock_aapl, text_df_appl = intersect_df(stock_aapl, text_df)
text_stats_appl = daily_stats(text_df_appl)
text_df_appl = daily_stats(text_df_appl)

twitter to emotion dataframe

feature list: 

              emotions        [std, mean, count],                                 --> no need to normalize
              
              stock           [open, high, low, close, volume],                   --> normalize percent change

              portfolio       [current money, stocks owned, stocks owned value],  --> keep track of portfolio

              date            [day of week, holiday, day of month, month]         --> engineer

              economic data   [NA]                                                --> normalize 

output: 

  volatility:

  best action

  predicted price


In [ ]:
# Process kroger data. 1) Merge global, stock, and company specific data into a timeseries, with zeros for missing data. 2) Split into train and test sets. 3) Scale the data. 4) Reshape the data into 3D for Resnet, 5) ensure data ordering and labels are correct

merged_df_kr = merge_data_timeline(kr_stats, stock_kr, text_stats_kr)
y_cols = ['open', 'high', 'low', 'close', 'volume']
x_cols = merged_df_kr.columns #  if x not in y_cols and x != 'date'... not removed because it is current day's data and we want to predict next day's data
X = merged_df_kr[x_cols]
Y = merged_df_kr[y_cols]
#delete the first row to allign the data
Y = Y.drop(Y.index[0]) # y is now the next day's data... e.g. we want to predict tomorrow's stock price
X = X.drop(X.index[-1]) 

# complete timeseries
Y = Y.values
X = X.values

# split into train and test sets by slicing
train_size = int(len(X) * 0.90)
test_size = len(X) - train_size

X_train, Y_train_kr = X[0:train_size,:], Y[0:train_size,:]
X_test, Y_test_kr = X[train_size:len(X),:], Y[train_size:len(Y),:]

dates_train = X_train[:,0]
dates_test = X_test[:,0]
X_train = X_train[:,1:].astype('float32')
X_test = X_test[:,1:].astype('float32')

# split dim 1 into 2
split_into = 2
X_train_kr = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_kr = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
#################################################################################################################################
# process apple data
merged_df_aapl = merge_data_timeline(aapl_stats, stock_aapl, text_stats_appl)
y_cols = ['open', 'high', 'low', 'close', 'volume']
x_cols = merged_df_kr.columns #  if x not in y_cols and x != 'date'... not removed because it is current day's data and we want to predict next day's data
X = merged_df_aapl[x_cols]
Y = merged_df_aapl[y_cols]
#delete the first row to allign the data
Y = Y.drop(Y.index[0]) # y is now the next day's data... e.g. we want to predict tomorrow's stock price
X = X.drop(X.index[-1]) 

# complete timeseries
Y = Y.values
X = X.values

# split into train and test sets by slicing
train_size = int(len(X) * 0.90)
test_size = len(X) - train_size

X_train, Y_train_aapl = X[0:train_size,:], Y[0:train_size,:]
X_test, Y_test_aapl = X[train_size:len(X),:], Y[train_size:len(Y),:]

dates_train = X_train[:,0]
dates_test = X_test[:,0]
X_train = X_train[:,1:].astype('float32')
X_test = X_test[:,1:].astype('float32')

# split dim 1 into 2
split_into = 2
X_train_aapl = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_aapl = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
window_Size = 32
X_train_win_kr = sliding_window_view_generator(X_train_kr, window_Size)
X_test_win_kr = sliding_window_view_generator(X_test_kr, window_Size)
Y_train_win_kr = sliding_window_view_generator(Y_train_kr, 1)
Y_test_win_kr = sliding_window_view_generator(Y_test_kr, 1)

X_train_win_aapl = sliding_window_view_generator(X_train_aapl, window_Size)
X_test_win_aapl = sliding_window_view_generator(X_test_aapl, window_Size)
Y_train_win_aapl = sliding_window_view_generator(Y_train_aapl, 1)
Y_test_win_aapl = sliding_window_view_generator(Y_test_aapl, 1)


model = model((1, window_Size, 216), 2, Y_test_win_kr.shape[1])

        
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()
num_epochs = 2
early_stopping = True



# move to gpu
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
best_loss = np.inf
best_epoch = 0
best_model_perams = None
patience = 10
hist = {'train_loss': [], 'test_loss': []}
# Train on kr data and test on aapl data
with tqdm(total=num_epochs) as pbar:
    for epoch in range(num_epochs):
        # Forward pass

        x = next(X_train_win_kr.next())
        y = next(Y_train_win_kr.next())
        predictions, actions  = model.forward(x)
        loss = model.action(actions, train=True)
        loss += criterion(predictions, y)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()

        hist['train_loss'].append(loss.item())
        pbar.set_description(f"Epoch [{epoch+1}/{num_epochs}], Loss: {hist['train_loss'][-1]:.4f}")
        pbar.update(1)

        # val step
        #if epoch % 10 == 0:
        with torch.no_grad():
            for _ in range(len(X_test_win_kr.x)//3):
                x = next(X_test_win_aapl.next())
                y = next(Y_test_win_aapl.next())
                predictions, actions = model.forward(x)
                loss += model.action(actions, train=False)
                loss += criterion(predictions, y)

            loss = loss / len(X_test_win_kr.x)
            hist['test_loss'].append(loss.item())

            #print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {hist['train_loss'][-1]:.4f}, Val Loss: {hist['test_loss'][-1]:.4f}")
            if loss.item() < best_loss:
                best_loss = loss.item()
                best_epoch = epoch
                best_model_perams = model.state_dict()
            else:
                if early_stopping and epoch - best_epoch > patience:
                    model.load_state_dict(best_model_perams)
                    print(f"Early stopping at epoch {epoch} with best loss {best_loss} at epoch {best_epoch}, history: {hist['test_loss'][-1]:.4f}")
                    break

In [ ]:
best_loss = np.inf
best_epoch = 0
best_model_perams = None
patience = 10
hist = {'train_loss': [], 'test_loss': []}

# trains on apple data and tests on kr data
with tqdm(total=num_epochs) as pbar:
    for epoch in range(num_epochs):
        # Forward pass
        x = next(X_train_win_aapl.next())
        y = next(Y_train_win_aapl.next())
        predictions, actions  = model.forward(x)
        loss = model.action(actions, train=True)
        loss += criterion(predictions, y)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()

        hist['train_loss'].append(loss.item())
        pbar.set_description(f"Epoch [{epoch+1}/{num_epochs}], Loss: {hist['train_loss'][-1]:.4f}")
        pbar.update(1)

        # val step
        #if epoch % 10 == 0:
        with torch.no_grad():
            for _ in range(len(X_test_win_kr.x)//3):
                x = next(X_test_win_kr.next())
                y = next(Y_test_win_kr.next())
                predictions, actions = model.forward(x)
                loss += model.action(actions, train=False)
                loss += criterion(predictions, y)

            loss = loss / len(X_test_win_kr.x)
            hist['test_loss'].append(loss.item())

            #print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {hist['train_loss'][-1]:.4f}, Val Loss: {hist['test_loss'][-1]:.4f}")
            if loss.item() < best_loss:
                best_loss = loss.item()
                best_epoch = epoch
                best_model_perams = model.state_dict()
            else:
                if early_stopping and epoch - best_epoch > patience:
                    model.load_state_dict(best_model_perams)
                    print(f"Early stopping at epoch {epoch} with best loss {best_loss} at epoch {best_epoch}, history: {hist['test_loss'][-1]:.4f}")
                    break